### 싱글모델로 분류

In [1]:

import torch 
import argparse
import yaml
import time
import multiprocessing as mp
import torch.nn.functional as F
from tabulate import tabulate
from tqdm import tqdm
from torch.utils.data import DataLoader
from pathlib import Path
#from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DistributedSampler, RandomSampler
from torch import distributed as dist
from nmc.models import *
from nmc.datasets import * 
from nmc.augmentations import get_train_augmentation, get_val_augmentation
from nmc.losses import get_loss
from nmc.schedulers import get_scheduler
from nmc.optimizers import get_optimizer
from nmc.utils.utils import fix_seeds, setup_cudnn, cleanup_ddp, setup_ddp
from tools.val import evaluate_epi
from nmc.utils.episodic_utils import * 
from scipy.cluster import hierarchy
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from torchvision import models
import torch.nn as nn
from torch.optim import lr_scheduler
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mutual_info_score
from scipy.cluster import hierarchy
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.utils.data import Subset
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import cv2

/root/anaconda3/envs/jm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('../configs/NMC.yaml') as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)
with open('../configs/APTOS.yaml') as f:
    cfg_apt = yaml.load(f, Loader=yaml.SafeLoader)
# print(cfg)
fix_seeds(3407)
setup_cudnn()
gpu = setup_ddp()
save_dir = Path(cfg['SAVE_DIR'])
save_dir.mkdir(exist_ok=True)
cleanup_ddp()

In [3]:
# Early Stopping
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_score):
        if self.best_score is None:
            self.best_score = val_score
        elif val_score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = val_score
            self.counter = 0
            
def get_train_augmentation(size):
    return transforms.Compose([
        transforms.Resize(size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.Lambda(lambda x: x.float() if x.dtype == torch.uint8 else x),
        transforms.Lambda(lambda x: x / 255.0 if x.max() > 1.0 else x),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

def get_val_test_transform(size):
    return transforms.Compose([
        transforms.Resize(size),
        transforms.Lambda(lambda x: x.float() if x.dtype == torch.uint8 else x),
        transforms.Lambda(lambda x: x / 255.0 if x.max() > 1.0 else x),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [4]:
import torch
import torch.nn as nn
from torchvision import models

class EfficientNetWithFeatures(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNetWithFeatures, self).__init__()
        # EfficientNet 모델 불러오기
        self.efficientnet = models.efficientnet_v2_m(pretrained=pretrained)
        
        # 분류기에 들어가는 최종 feature 크기
        num_ftrs = self.efficientnet.classifier[1].in_features
        
        # 새로운 분류기 정의
        self.efficientnet.classifier = nn.Sequential(
            nn.BatchNorm1d(num_ftrs),
            nn.Linear(num_ftrs, 1)
        )

    def forward(self, x):
        # EfficientNet의 forward 함수 호출하여 feature 추출
        features = self.efficientnet.features(x)
        features = self.efficientnet.avgpool(features)
        features = torch.flatten(features, 1)
        
        # Flatten된 feature를 classifier로 전달
        out = self.efficientnet.classifier(features)
        
        # 최종 feature와 분류기의 출력 반환
        return features, out

In [6]:
start = time.time()
best_mf1 = 0.0
device = torch.device(cfg['DEVICE'])
print("device : ", device)
num_workers = mp.cpu_count()
train_cfg, eval_cfg = cfg['TRAIN'], cfg['EVAL']
dataset_cfg, model_cfg = cfg['DATASET'], cfg['MODEL']
loss_cfg, optim_cfg, sched_cfg = cfg['LOSS'], cfg['OPTIMIZER'], cfg['SCHEDULER']
epochs, lr = train_cfg['EPOCHS'], optim_cfg['LR']

model_name = 'efficientnet'
efficientnet = EfficientNetWithFeatures(pretrained=True).to(device)

target_label = 0
image_dir = Path(dataset_cfg['ROOT']) / 'train_images'
image_size = [256, 256]
batch_size = 8
dir1 = 'img'+str(image_size[0])
dir2 = 'batch'+str(batch_size)
model_dir = f'models/{model_name}/{dir1}/{dir2}/best_model'
train_transform = get_train_augmentation(image_size)
val_test_transform = get_val_test_transform(image_size)

dataset = eval(dataset_cfg['NAME'])(
    dataset_cfg['ROOT'] + '/cropped_images_1424x1648',
    dataset_cfg['TRAIN_RATIO'],
    dataset_cfg['VALID_RATIO'],
    dataset_cfg['TEST_RATIO'],
    transform=None,
    target_label=target_label
)
trainset, valset, testset = dataset.get_splits()
trainset.transform = train_transform
valset.transform = val_test_transform
testset.transform = val_test_transform

trainloader = DataLoader(trainset, batch_size=batch_size, num_workers=num_workers, drop_last=True, pin_memory=True)
valloader = DataLoader(valset, batch_size=1, num_workers=1, pin_memory=True)
testloader = DataLoader(testset, batch_size=1, num_workers=1, pin_memory=True)

# dataset_apt_cfg = cfg_apt['DATASET']
# dataset_apt = eval(dataset_apt_cfg['NAME'])(
#     dataset_apt_cfg['ROOT'] + '/combined_images',
#     dataset_apt_cfg['TRAIN_RATIO'],
#     dataset_apt_cfg['VALID_RATIO'],
#     dataset_apt_cfg['TEST_RATIO'],
#     transform=None,
#     target_label=target_label
# )
# trainset_apt, valset_apt, testset_apt = dataset_apt.get_splits()
# trainset_apt.transform = train_transform
# valset_apt.transform = val_test_transform
# testset_apt.transform = val_test_transform

# trainloader_apt = DataLoader(trainset_apt, batch_size=batch_size, num_workers=num_workers, drop_last=True, pin_memory=True)
# valloader_apt = DataLoader(valset_apt, batch_size=1, num_workers=1, pin_memory=True)
# testloader_apt = DataLoader(testset_apt, batch_size=1, num_workers=1, pin_memory=True)


device :  cuda:1


/root/anaconda3/envs/jm/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/anaconda3/envs/jm/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_M_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


/data/nmc/processed_image/cropped_images_1424x1648
Target label: 0
Train size: 4533, Positive samples: 1935
Validation size: 971, Positive samples: 415
Test size: 972, Positive samples: 415


In [7]:
# L2 regularization
weight_decay = 1e-4
optimizer = torch.optim.AdamW(efficientnet.parameters(), lr=0.0001, weight_decay=weight_decay)
criterion = nn.BCEWithLogitsLoss()
scaler = GradScaler(enabled=train_cfg['AMP'])
# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)


/tmp/ipykernel_282674/34482705.py:5: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=train_cfg['AMP'])
/root/anaconda3/envs/jm/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [8]:
def train_epoch(model, dataloader, criterion, optimizer, scaler, device):
    model.train()
    total_loss = 0
    for images, labels in tqdm(dataloader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        with autocast(enabled=scaler is not None):
            features, outputs = model(images)
            loss = criterion(outputs.squeeze(), labels.float())
        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

def evaluate(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            
            features, outputs = model(images)
            preds = (torch.sigmoid(outputs) > 0.5).int().squeeze()
            
            if preds.dim() == 0:
                preds = preds.unsqueeze(0)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    f1 = f1_score(all_labels, all_preds)
    return f1

def train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, scaler, device, epochs, model_dir):
    best_f1 = 0.0
    early_stopping = EarlyStopping(patience=10, min_delta=0.001)
    
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        train_loss = train_epoch(model, train_loader, criterion, optimizer, scaler, device)
        val_f1 = evaluate(model, val_loader, device)
        
        print(f"Training Loss: {train_loss:.4f}")
        print(f"Validation F1 Score: {val_f1:.4f}")
        
        scheduler.step(val_f1)
        
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), f'{model_dir}_{target_label}.pth')
            print("New best model saved!")
        
        early_stopping(val_f1)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
        
        print()
    
    return best_f1

In [ ]:
# weight_decay = 1e-5
# optimizer = torch.optim.AdamW(efficientnet.parameters(), lr=0.00001, weight_decay=weight_decay)
# criterion = nn.BCEWithLogitsLoss()
# scaler = GradScaler(enabled=train_cfg['AMP'])
# # Learning rate scheduler
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)
# efficientnet.load_state_dict(torch.load('/workspace/jhmoon/NMCS/models/efficientnet/img256/batch8/best_model_0.pth'))
# # Main execution code
# # 정규화, lr스케쥴링, 데이터 증강, 조기종료, 배치정규화
# epochs = 100
# best_f1 = train_and_evaluate(efficientnet, trainloader_apt, valloader_apt, criterion, optimizer, scaler, device, epochs, model_dir)

# print(f"Training completed. Best F1 Score: {best_f1:.4f}")

# # Final evaluation on test set
# efficientnet.load_state_dict(torch.load(f'{model_dir}_{target_label}.pth'))
# test_f1 = evaluate(efficientnet, testloader_apt, device)
# print(f"Test F1 Score: {test_f1:.4f}")

In [11]:
# Main execution code
# 정규화, lr스케쥴링, 데이터 증강, 조기종료, 배치정규화
epochs = 100
best_f1 = train_and_evaluate(efficientnet, trainloader, valloader, criterion, optimizer, scaler, device, epochs, model_dir)

print(f"Training completed. Best F1 Score: {best_f1:.4f}")

# Final evaluation on test set
efficientnet.load_state_dict(torch.load('model/best_model_{}.pth'.format(target_label)))
test_f1 = evaluate(efficientnet, testloader, device)
print(f"Test F1 Score: {test_f1:.4f}")

Epoch 1/100


Training:   0%|          | 0/566 [00:00<?, ?it/s]/tmp/ipykernel_282674/4026436473.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=scaler is not None):
Evaluating: 100%|██████████| 971/971 [01:26<00:00, 11.17it/s]


Training Loss: 0.4137
Validation F1 Score: 0.8345
New best model saved!

Epoch 2/100


Training:   0%|          | 0/566 [00:00<?, ?it/s]/tmp/ipykernel_282674/4026436473.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=scaler is not None):
Evaluating: 100%|██████████| 971/971 [01:40<00:00,  9.66it/s]


Training Loss: 0.3618
Validation F1 Score: 0.8458
New best model saved!

Epoch 3/100


...

Training:   0%|          | 0/566 [00:00<?, ?it/s]/tmp/ipykernel_282674/4026436473.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=scaler is not None):
Evaluating: 100%|██████████| 971/971 [01:14<00:00, 13.07it/s]


Training Loss: 0.0077
Validation F1 Score: 0.8765

Epoch 43/100


Training:   0%|          | 0/566 [00:00<?, ?it/s]/tmp/ipykernel_282674/4026436473.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=scaler is not None):
Evaluating: 100%|██████████| 971/971 [01:13<00:00, 13.17it/s]
/tmp/ipykernel_282674/3897559536.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializati

Training Loss: 0.0077
Validation F1 Score: 0.8822
Early stopping triggered
Training completed. Best F1 Score: 0.8833


FileNotFoundError: [Errno 2] No such file or directory: 'model/best_model_0.pth'

In [22]:
def denormalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

def evaluate_and_save_misclassified(model, dataloader, device, save_dir):
    model.eval()
    all_preds = []
    all_labels = []
    misclassified = []
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # 정규화에 사용된 평균과 표준편차 (이 값들은 사용한 transform에 맞게 조정해야 함)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(dataloader, desc="Evaluating")):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            preds = (torch.sigmoid(outputs) > 0.5).int().squeeze()
            
            if preds.dim() == 0:
                preds = preds.unsqueeze(0)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            # 잘못 분류된 이미지 저장
            for j, (pred, label) in enumerate(zip(preds, labels)):
                if pred != label:
                    img = images[j].cpu()
                    
                    # 정규화 해제
                    img = denormalize(img, mean, std)
                    
                    # [0, 1] 범위의 float 텐서를 [0, 255] 범위의 uint8로 변환
                    img = (img * 255).byte()
                    
                    img = transforms.ToPILImage()(img)
                    img_name = f"misclassified_{i}_{j}_pred{pred.item()}_true{label.item()}.png"
                    img.save(os.path.join(save_dir, img_name))
                    misclassified.append((i, j, pred.item(), label.item()))
    
    return all_preds, all_labels, misclassified

In [23]:
# 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Model definition (changed to binary classification)
efficientnet = models.efficientnet_v2_m(pretrained=True)
num_ftrs = efficientnet.classifier[1].in_features
efficientnet.classifier = nn.Sequential(
    nn.BatchNorm1d(num_ftrs),
    nn.Linear(num_ftrs, 1)
)
efficientnet.load_state_dict(torch.load('model/best_model_{}.pth'.format(target_label), map_location=device))
efficientnet = efficientnet.to(device)
efficientnet.eval()


/root/anaconda3/envs/gm/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/root/anaconda3/envs/gm/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_M_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [33]:
import os
from torchvision import transforms
# 평가 및 잘못 분류된 이미지 저장
save_dir = "misclassified_images"
all_preds, all_labels, misclassified = evaluate_and_save_misclassified(efficientnet, testloader, device, save_dir)

print(f"Total misclassified images: {len(misclassified)}")
print("Misclassified images saved in:", save_dir)

# 잘못 분류된 이미지 분석
false_positives = sum(1 for pred, label in zip(all_preds, all_labels) if pred == 1 and label == 0)
false_negatives = sum(1 for pred, label in zip(all_preds, all_labels) if pred == 0 and label == 1)

print(f"False Positives: {false_positives}")
print(f"False Negatives: {false_negatives}")

# 몇 개의 잘못 분류된 이미지 예시 출력
print("\nSome examples of misclassified images:")
for i, (batch, index, pred, true) in enumerate(misclassified[:5]):  # 처음 5개만 출력
    print(f"Image {i+1}: Batch {batch}, Index {index}, Predicted {pred}, True label {true}")

Evaluating:   0%|          | 0/146 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 146/146 [00:07<00:00, 18.99it/s]

Total misclassified images: 16
Misclassified images saved in: misclassified_images
False Positives: 10
False Negatives: 6

Some examples of misclassified images:
Image 1: Batch 1, Index 0, Predicted 1, True label 0.0
Image 2: Batch 4, Index 0, Predicted 1, True label 0.0
Image 3: Batch 13, Index 0, Predicted 1, True label 0.0
Image 4: Batch 19, Index 0, Predicted 1, True label 0.0
Image 5: Batch 30, Index 0, Predicted 1, True label 0.0


In [34]:
def denormalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.hooks = []
        self.gradients = None
        self.activations = None
        
        self._register_hooks()
        
    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output
        
        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0]
        
        target_layer = self.target_layer
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)
    
    def generate_cam(self, input_image, target_class=None):
        self.model.zero_grad()
        output = self.model(input_image)
        
        if target_class is None:
            target_class = 0  # 이진 분류에서는 항상 0 클래스에 대해 계산
        
        output = torch.sigmoid(output)
        output[0][target_class].backward()
        
        pooled_gradients = torch.mean(self.gradients, dim=[0, 2, 3])
        for i in range(self.activations.shape[1]):
            self.activations[:, i, :, :] *= pooled_gradients[i]
        
        heatmap = torch.mean(self.activations, dim=1).squeeze().detach().cpu().numpy()
        heatmap = np.maximum(heatmap, 0)
        heatmap /= np.max(heatmap)
        
        return heatmap

In [35]:
def apply_gradcam(model, image, target_class=None):
    # ResNext의 마지막 컨볼루션 레이어를 타겟으로 설정
    target_layer = model.features[-1]
    
    grad_cam = GradCAM(model, target_layer)
    
    # 이미지가 이미 전처리되었다고 가정
    input_tensor = image.unsqueeze(0).to(device)
    input_tensor.requires_grad = True
    
    # GradCAM 생성
    cam = grad_cam.generate_cam(input_tensor, target_class)
    
    # 원본 이미지로 변환 (역정규화 포함)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    orig_img = denormalize(image.clone(), mean, std).permute(1, 2, 0).cpu().numpy()
    orig_img = np.clip(orig_img, 0, 1)
    
    # 히트맵 크기 조정
    cam_resized = cv2.resize(cam, (orig_img.shape[1], orig_img.shape[0]))
    
    # 히트맵을 RGB로 변환
    heatmap = cv2.applyColorMap(np.uint8(255 * cam_resized), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    
    # 히트맵과 원본 이미지 합성
    superimposed_img = heatmap * 0.4 + orig_img * 255
    superimposed_img = superimposed_img / np.max(superimposed_img)
    
    return orig_img, heatmap / 255.0, superimposed_img

In [36]:
def apply_gradcam_to_misclassified(model, testloader, device, save_dir, num_images=10):
    model.eval()  # 평가 모드로 설정
    processed_count = 0
    
    for batch_idx, (inputs, labels) in enumerate(testloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 기울기 계산 활성화
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            predicted = (torch.sigmoid(outputs) > 0.5).int().squeeze()
            
        if predicted.dim() == 0:
            predicted = predicted.unsqueeze(0)
        
        # 오분류된 이미지 찾기
        misclassified = (predicted != labels).nonzero(as_tuple=True)[0]
        
        for idx in misclassified:
            img = inputs[idx]
            pred = predicted[idx].item()
            true = labels[idx].item()
            
            # GradCAM 적용
            orig_img, heatmap, gradcam_img = apply_gradcam(model, img, target_class=0)
            
            # 결과 저장
            fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
            
            ax1.imshow(orig_img)
            ax1.set_title(f"Original: True {true}, Pred {pred}")
            ax1.axis('off')
            
            ax2.imshow(heatmap)
            ax2.set_title("Heatmap")
            ax2.axis('off')
            
            ax3.imshow(gradcam_img)
            ax3.set_title("GradCAM")
            ax3.axis('off')
            
            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, f"gradcam_{batch_idx}_{idx}_pred_{pred}_true_{true}.png"))
            plt.close()
            

    print(f"Processed {processed_count} misclassified images.")

In [37]:
# GradCAM 적용
apply_gradcam_to_misclassified(efficientnet, testloader, device, "gradcam_results")
print("GradCAM images saved in: gradcam_results")

Processed 0 misclassified images.
GradCAM images saved in: gradcam_results
